In [6]:
# Authors: J. Schattschneider, U. Z ̈olzer
# Adapted by: Wallace Abreu
# 
# shows short-time spectra of signal, starting
# at k=start, with increments of STEP with N-point FFT
# dynamic range from -baseplane in dB up to 20*log(clippingpoint)
# in dB versus time axis

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib import colors as mcolors
import matplotlib.ticker as ticker

In [47]:
def waterfspec(signal, steps, start=0, N=1024, fs=48000, clippingpoint=0, baseplane=-100):
    # Signal processing section
    window = np.blackman(N)
    window = window * N / sum(window)
    n = len(signal)
    rest = n - start - N
    nos = int(np.round(rest / steps))
    speks = np.zeros((nos, N + 1))
    if nos > rest / steps:
         nos = nos - 1
    x = np.linspace(0, fs / 1000, N + 1)
    z = np.zeros_like(x)
    cup = z + clippingpoint
    cdown = z + baseplane
    signal = signal + 0.0000001
    z_values = np.zeros((len(x), nos))
    y = np.linspace(0, nos, nos)
    for i in range(nos):
        spek1 = 20 * np.log10(np.abs(np.fft.fft(window * signal[start + i * steps : start + N + i * steps])) / N / 0.5)
        spek = np.concatenate(([-200], spek1[0 : N])).T
        spek = (spek > cup.T) * cup.T + (spek <= cup.T) * spek
        spek = (spek < cdown.T) * cdown.T + (spek >= cdown.T) * spek
        spek[0] = baseplane - 10
        spek[N//2 - 1] = baseplane - 10
        z_values[:, i] = spek
        speks[i, :] = spek

    # Graphics section    

    spec_len = N // 2

    fig = plt.figure()

    ax = fig.add_subplot(111, projection='3d')
    x = list(range(spec_len))
    for i in range(nos):
        y = [i] * spec_len
        z = speks[i, 0 : spec_len].tolist()
        vertices = [list(zip(x, y, z))]

        poly = Poly3DCollection(vertices, facecolors='w', linewidths=1, alpha=0.8)
        poly.set_edgecolor('k')
        ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1, 1.5, 0.5, 1]))
        ax.add_collection3d(poly)

    ax.add_collection3d(poly)

    ax.set_xlim(0, spec_len)

    ax.set_ylim(0, nos)

    ax.set_zlim(np.min(speks), np.max(speks))

    ax.xaxis.set_major_locator(plt.MaxNLocator(5))

    xticks_list = [str(round(f, 1)) for f in np.linspace(0, fs / 2000, 5)]
    ax.set_xticklabels(xticks_list)

    ax.yaxis.set_major_locator(plt.MaxNLocator(8))

    yticks_list = [n for n in np.arange(0, len(signal), 1000)]
    ax.set_yticklabels(yticks_list)

    ax.set_xlabel(r'f in kHz $\to$')

    ax.set_ylabel(r'n $\longrightarrow$ ')

    ax.set_zlabel(r'Magnitude in dB $\to$')
    